## Idea Trabajo Práctico Final: Predicción de Resultado de Partidos de Tenis 
El objetivo de la red neuronal a implementar es poder predecir los resultados de cierta temporada de tenis actual considerando el historial de partidos de temporadas pasadas, como así también el historial individual entre los jugadores de cada partido. 
### Dataset de Referencia
El dataset de referencia será el historial de partidos de la Asociación de Tenistas Profesionales (ATP) brindado en su base de datos. Este no sólo incluye quien ha ganado cada partido, sino el puntaje y los porcentajes de las características relevantes de cada jugada en particular ('primeros servicios', 'segundos servicios', 'aces', 'porcentaje de devoluciones', 'doble faltas', etc.).
Por lo tanto, el set de entrenamiento de la red consistirá de un vector de características de entrada (X) representando las características de los jugadores involucrados y del partido, y un vector de salida correspondiente que determina que jugador gana. 
A continuación se realiza el parseo de datos:


In [26]:
pd_atp_matches_2017.iloc[0]

tourney_id                  2017-M020
tourney_name                 Brisbane
surface                          Hard
draw_size                          32
tourney_level                       A
tourney_date                 20170102
match_num                         300
winner_id                      105777
winner_seed                         7
winner_entry                      NaN
winner_name           Grigor Dimitrov
winner_hand                         R
winner_ht                         188
winner_ioc                        BUL
winner_age                    25.6345
winner_rank                        17
winner_rank_points               2035
loser_id                       105453
loser_seed                          3
loser_entry                       NaN
loser_name              Kei Nishikori
loser_hand                          R
loser_ht                          178
loser_ioc                         JPN
loser_age                     27.0116
loser_rank                          5
loser_rank_p

In [40]:
import numpy as np
import pandas as pd

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
pd_atp_matches_2017 = pd.read_csv('tennis_atp/atp_matches_2017.csv')
my_data_stats = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon']
pd_atp_matches_2017=pd_atp_matches_2017.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches_2017=pd.merge(pd_atp_matches_2017,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches_2017=pd_atp_matches_2017.drop_duplicates(keep='first').reset_index(drop=True)

Debajo se muestra un ejemplo del dataset scores obtenido:

In [28]:
#pd_atp_matches_2017.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]
pd.set_option('display.max_rows', 5000)

pd_atp_matches_2017

,tourney_id,tourney_name,round,winner_id,winner_name,loser_id,loser_name,w_ace,l_ace,w_df,...,w_svpt,w_1stWon,w_1stIn,l_svpt,l_1stWon,l_1stIn,w_ret_w,w_ret_tot,l_ret_w,l_ret_tot
0,2017-M020,Brisbane,F,105777,Grigor Dimitrov,105453,Kei Nishikori,7,4,2,...,77,41,52,69,36,49,24,69,24,77
1,2017-M020,Brisbane,SF,105777,Grigor Dimitrov,105683,Milos Raonic,4,4,1,...,58,27,36,61,24,28,21,61,13,58
2,2017-M020,Brisbane,SF,105453,Kei Nishikori,104527,Stanislas Wawrinka,1,9,1,...,77,37,56,61,27,37,24,61,26,77
3,2017-M020,Brisbane,QF,105683,Milos Raonic,104745,Rafael Nadal,23,4,3,...,97,50,62,84,39,61,31,84,31,97
4,2017-M020,Brisbane,QF,105777,Grigor Dimitrov,106233,Dominic Thiem,3,6,3,...,94,42,52,82,29,37,29,82,29,94
5,2017-M020,Brisbane,QF,105453,Kei Nishikori,111442,Jordan Thompson,3,1,0,...,34,18,19,47,15,28,27,47,6,34
6,2017-M020,Brisbane,QF,104527,Stanislas Wawrinka,106378,Kyle Edmund,11,2,3,...,119,47,67,97,52,65,34,97,44,119
7,2017-M020,Brisbane,R16,105683,Milos Raonic,106043,Diego Sebastian Schwartzman,12,0,1,...,53,30,40,44,17,29,21,44,16,53
8,2017-M020,Brisbane,R16,104745,Rafael Nadal,104999,Mischa Zverev,1,2,2,...,38,15,18,38,13,25,22,38,8,38
9,2017-M020,Brisbane,R16,106233,Dominic Thiem,105032,Samuel Groth,11,11,1,...,65,36,44,58,34,35,15,58,14,65


In [77]:
display(pd_atp_matches_2017.loc[0])

tourney_id                  2017-M020
tourney_name                 Brisbane
round                               F
winner_id                      105777
winner_name           Grigor Dimitrov
loser_id                       105453
loser_name              Kei Nishikori
winner_rank                        17
winner_rank_points               2035
loser_rank                          5
loser_rank_points                4905
w_ace                               7
l_ace                               4
w_df                                2
l_df                                0
w_svpt                             77
w_1stWon                           41
w_1stIn                            52
l_svpt                             69
l_1stWon                           36
l_1stIn                            49
w_2ndWon                           12
l_2ndWon                            9
w_ret_w                            24
w_ret_tot                          69
l_ret_w                            24
l_ret_tot   

## Extracción de Features 
A continuación se describiran la extracción de features de cada partido. 

A continuación se busca la lista de todos los jugadores presentes.

In [ ]:
pd_atp_matches_2017=pd_atp_matches_2017.assign(WRP=0)

for i in range(0,len(pd_atp_matches_2017.index)):
    pd_atp_matches_2017.loc[i, 'WRP'] = getWRPi(pd_atp_matches_2017.loc[i, 'winner_id'],pd_atp_matches_2017.loc[i, 'loser_id'])


In [30]:
all_winner_players=pd_atp_matches_2017.filter(['winner_id','winner_name'], axis=1)
all_winner_players.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)

all_loser_players=pd_atp_matches_2017.filter(['loser_id','loser_name'], axis=1)
all_loser_players.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)

#print(all_winner_players)
#print(all_loser_players)

all_players=all_winner_players.append(all_loser_players, ignore_index=True).drop_duplicates()
all_players
all_players.sort_values(by=['ID']).reset_index(drop=True);
#players_list=all_players.ID.unique()
#players_list.sort()
#players_list

In [76]:
def getWRP(id1,id2):

    #id1 = 105223 #delpo
    #id2 = 105992 #Ryan harrison

    #id1 = 104229 #nadal
    #id2 = 105032 #groth
    array=[id1,id2]
    locate_1=all_players.loc[(all_players['ID'].isin([array[0]]))]
    locate_2=all_players.loc[(all_players['ID'].isin([array[1]]))]

    #print('El jugador #1 es: ' + locate_1.iloc[0,1])
    #print('El jugador #2 es: ' + locate_2.iloc[0,1])

    #print('En el 2017 se enfrentaron a: ' )

    opponents_p1_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[0]]))]
    opponents_p1_1=opponents_p1_1[['loser_id','loser_name']]
    opponents_p1_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p1_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[0]]))]
    opponents_p1_2=opponents_p1_2[['winner_id','winner_name']]
    opponents_p1_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p1=opponents_p1_1.append(opponents_p1_2, ignore_index=True).drop_duplicates()

    opponents_p2_1=pd_atp_matches_2017.loc[(pd_atp_matches_2017['winner_id'].isin([array[1]]))]
    opponents_p2_1=opponents_p2_1[['loser_id','loser_name']]
    opponents_p2_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p2_2=pd_atp_matches_2017.loc[(pd_atp_matches_2017['loser_id'].isin([array[1]]))]
    opponents_p2_2=opponents_p2_2[['winner_id','winner_name']]
    opponents_p2_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p2=opponents_p2_1.append(opponents_p2_2, ignore_index=True).drop_duplicates()

    common_opponents = pd.merge(opponents_p1, opponents_p2, how='inner', on=['ID']).filter(['ID','NAME_x'], axis=1)
    common_opponents=common_opponents.assign(WRP1=0,WRP2=0)

    for i in range(0,len(common_opponents.index)):
        common_opponents.loc[i, 'WRP1'] = getWRPi(id1,common_opponents.loc[i, 'ID'])
        common_opponents.loc[i, 'WRP2'] = getWRPi(id2,common_opponents.loc[i, 'ID'])

    WRP1=common_opponents['WRP1'].mean()
    WRP2=common_opponents['WRP2'].mean()
    WRP=WRP1-WRP2
    #print(WRP)

    #display_side_by_side(opponents_p1,opponents_p2,common_opponents)
    return(WRP)

In [75]:
#this function returns WRP of a player against a certain adversary

def getWRPi(myopponent,adversary):
    exp1=pd_atp_matches_2017['winner_id'].isin([myopponent])
    exp2=pd_atp_matches_2017['loser_id'].isin([adversary])
    exp3=pd_atp_matches_2017['loser_id'].isin([myopponent])
    exp4=pd_atp_matches_2017['winner_id'].isin([adversary])

    matches_won=pd_atp_matches_2017.loc[(exp1&exp2)]
    matches_won=matches_won.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    matches_lost=pd_atp_matches_2017.loc[(exp3&exp4)]
    matches_lost=matches_lost.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','w_ret_w','w_ret_tot','l_ret_w','l_ret_tot'], axis=1)

    matches_won['WRP'] = matches_won['w_ret_w']/matches_won['w_ret_tot']
    matches_lost['WRP'] = matches_lost['l_ret_w']/matches_lost['l_ret_tot']
    matches_won=matches_won.append(matches_lost)
    matches_won
    return matches_won['WRP'].mean()

In [41]:
results = pd_atp_matches_2017.filter(['winner_id','loser_id'], axis=1)
results['RANK'] = pd_atp_matches_2017['winner_rank'] - pd_atp_matches_2017['loser_rank']
results['POINTS'] = pd_atp_matches_2017['winner_rank_points'] - pd_atp_matches_2017['loser_rank_points']
results['FS'] = pd_atp_matches_2017['w_1stIn'] - pd_atp_matches_2017['l_1stIn']
results['W1SP'] = pd_atp_matches_2017['w_1stWon'] - pd_atp_matches_2017['l_1stWon']
results['W2SP'] = pd_atp_matches_2017['w_2ndWon'] - pd_atp_matches_2017['l_2ndWon']

WSP1 = pd_atp_matches_2017['w_1stWon'] * pd_atp_matches_2017['w_1stIn'] + pd_atp_matches_2017['w_2ndWon'] * (1-pd_atp_matches_2017['w_1stIn'])
WSP2 = pd_atp_matches_2017['l_1stWon'] * pd_atp_matches_2017['l_1stIn'] + pd_atp_matches_2017['l_2ndWon'] * (1-pd_atp_matches_2017['l_1stIn'])
results['WSP'] = WSP1 - WSP2

results.rename(columns={'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

,ID1,ID2,RANK,POINTS,FS,W1SP,W2SP,WSP
0,105777,105453,12.0,-2870.0,3,5,3.0,188
1,105777,105683,14.0,-3415.0,8,3,2.0,102
2,105453,104527,1.0,-410.0,19,10,4.0,663
3,105683,104745,-6.0,2150.0,1,11,2.0,585
4,105777,106233,9.0,-1380.0,15,13,-1.0,802
5,105453,111442,-74.0,4216.0,-9,3,5.0,-123
9,106233,105032,-172.0,3095.0,9,2,6.0,55
1000,104925,104597,-74.0,6410.0,8,1,2.0,151
1001,105657,104198,-20.0,70.0,0,2,-11.0,925
1002,106432,104999,28.0,-344.0,-19,-11,17.0,-1777


Los datos luego serán seleccionados partido a partido, a partir del dataset obtenido. Se realiza la siguiente división:

| Categoría  |  Detalle | 
|---|---|
| ID1 |  ID Jugador 1 |
| ID2 |  ID Jugador 2 |
| RANK |  Ranking ATP |
| POINTS  |  Puntos ATP | 
| FS | Porcentaje  de primeros servicios |
| W1SP | Porcentaje de primeros servicios ganados |
| W2SP | Porcentaje de segundos servicios ganados |
| WSP | Porcentaje de servicios ganados globales |
| WRP | Porcentaje de devolución ganada |
| TPW | Porcentaje de puntos totales ganados |
| TMW | Porcentaje de todos los partidos ganados |
| ACES | Promedio de aces por game |
| DF | Promedio de doble faltas por game |
| UE | Promedio de errores no forzados por game |
| WIS | Promedio de winners por game |
| BP | Porcentaje de break points ganados |
| NA | Porcentaje de puntos en la red ganados |
| A1S | Promedio de velocidad primer servicio |
| A2S | Promedio de velocidad de segundo servicio |
| FATIGUE | Fatiga por partidos jugados en 3 dias anteriores |
| RETIRED | Indica si es el primer partido luego de un retiro |
| DIRECT | Ventaja H2H entre los jugadores |

Cada partido tiene estas características que suelen indicar las diferencias entre ambos jugadores. Por lo que por ejemplo, para ATPRank, si RANK1 es el ranking del jugador 1 y RANK2 es el ranking del jugador 2, RANK = RANK1 - RANK2.
Para el entrenamiento, cada partido tendrá como salida un vector que inidque que jugador gana o que jugador pierde. Es decir, 1 si el jugador ganó o 0 si el jugador perdió.

Finalmente, el dataset será dividido en tres partes:
- Set de entrenamiento (2008 - 2014)
- Set de validación  (2015 - 2016)
- Set de prueba (2017-2018)



### Soluciones Propuestas
Se propone evaluar si utilizar una red multicapa en el entorno tensorflow para la implementación del proyecto (el cual se entrena con todo el dataset) o una red neuronal recurrente la cual posee una memoria 'selectiva' de partidos anteriores. Estas soluciones están sujetas a cambiar de acuerdo a las nuevas técnicas a aprender en la materia.

### Medición de la Calidad de la Solución Obtenida
La solución obtenida se limitará a predecir el ganador de cada partido. Esto tiene como consecuencia determinar el resultado de cada uno de los torneos a nivel ATP de la temporada, como así también el resultado del ranking al finalizar la temporada. Se contrastaran con los resultados reales, pretendiéndose como mínimo una efectividad superior al 50%.

### Proyectos Similares Implementados en Internet
El proyecto de referencia será la tesis presente en el siguiente link: https://www.doc.ic.ac.uk/teaching/distinguished-projects/2015/m.sipko.pdf el cuál plantea la predicción de partidos de tenis con diversos métodos incluyendo Machine Learning y mostrando sus resultados. Otro proyecto más breve se encuentra en el siguiente link: http://deepakn94.github.io/assets/papers/6.867.pdf. La técnica para solucionar el problema también puede referirse a predictores de otros deportes.
